# Tensorflow Ops

## Visualize with TensorBoard
Go to terminal, run:
```
$ python3 [yourprogram].py
$ tensorboard --logdir="./graphs" --port 6006
```
Then open your browser and go to: http://localhost:6006/

Explicitly name the variables

In [1]:
import tensorflow as tf

a = tf.constant(2, name='a')
b = tf.constant(3, name='b')
x = tf.add(a, b, name='add')
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
    print(sess.run(x))
writer.close()

5


## Constants, Sequences, Variables, Ops

### constants 
```
tf.constant(
    value,
    dtype=None,
    shape=None,
    name='Const',
    verify_shape=False
)
a = tf.cosntant([2,2], name='a')
```
specific values
```
tf.zeros(shape, dtype=tf.float32, name=None)
tf.zeros([2,3], tf.int32)
tf.zeros_like(input_tensor) # creates a tensor of same shape/type but with all zeros

tf.ones(shape, dtype=tf.float32, name=None)
tf.ones_like(input_tensor, dtype=None, name=None, optimize=True)

tf.fill(dims, value, name=None) # creates a tensor filled with a scalar value
```
Constants as sequences
```
tf.lin_space(start, stop, num, name=None)
tf.range(start, limit=None, delta=1, dtype=None, name='range) # not iterable
```
Random constants
```
tf.set_random_seed(seed)
tf.random_normal
tf.truncated_normal
tf.random_uniform
tf.random_shuffle
tf.random_crop
tf.multinomial
tf.random_gamma
```

### Operations
Similar to numpy

Wizard of Div
```
a = tf.constant([2, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')
with tf.Session() as sess:
	print(sess.run(tf.div(b, a)))             ⇒ [[0 0] [1 1]]
	print(sess.run(tf.divide(b, a)))          ⇒ [[0. 0.5] [1. 1.5]]
	print(sess.run(tf.truediv(b, a)))         ⇒ [[0. 0.5] [1. 1.5]]
	print(sess.run(tf.floordiv(b, a)))        ⇒ [[0 0] [1 1]]
	print(sess.run(tf.realdiv(b, a)))         ⇒ # Error: only works for real values
	print(sess.run(tf.truncatediv(b, a)))     ⇒ [[0 0] [1 1]]
	print(sess.run(tf.floor_div(b, a)))       ⇒ [[0 0] [1 1]]

```

### Data Types
Boolean, numeric(int,float), strings

Use TF DType when possible
* Python native types: TensorFlow has to infer Python type
* NumPy arrays: NumPy is not GPU compatible

### What's wrong with constants?
* constants are stored in the graph definition
* This makes loading graphs expensive when constants are big

Only use constants for primitive types

Use variables or readers for more data that requires mroe memory

### Variables
Definition:

In [2]:
# With tf.Variable:
s1 = tf.Variable(2, name="scalar")
m1 = tf.Variable([[0,1], [2,3]], name="matrix")
W1 = tf.Variable(tf.zeros([784,10]))

# With tf.get_variable (better)
s = tf.get_variable("scalar", initializer=tf.constant(2))
m = tf.get_variable("matrix", initializer=tf.constant([[0,1],[2,3]]))
W = tf.get_variable("big_matrix", shape=(784,10), initializer=tf.zeros_initializer())

tf.Variable holds serveral ops:
```
x = tf.Variable(...)

x.initializer # init op
x.value() # read op
x.assign(...) # write op
x.assign_add(...) # and more
```

Initialize your variables:

In [3]:
# initialize all variables
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

# initialize subset of variables:
with tf.Session() as sess:
    sess.run(tf.variables_initializer([s,m]))
    
# initialize a single variable:
with tf.Session() as sess:
    sess.run(W.initializer)

eval() a variable:

In [4]:
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W)
    print(W.eval())

<tf.Variable 'big_matrix:0' shape=(784, 10) dtype=float32_ref>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


assign() a variable:

In [5]:
# assign()
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    sess.run(assign_op)
    print(W.eval())
print("\n")

# assign() runs every time
my_var = tf.Variable(2, name="my_var")
my_var_times_two = my_var.assign(2 * my_var)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    print(my_var.eval())
    sess.run(my_var_times_two)
    print(my_var.eval())
    sess.run(my_var_times_two)
    print(my_var.eval())
print("\n")

# assign_add(), assign_sub()
my_var = tf.Variable(10)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    sess.run(my_var.assign_add(10))
    print(my_var.eval())
    sess.run(my_var.assign_sub(2))
    print(my_var.eval())
print("\n")
    
# each session maintain its copy of variables
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()
sess1.run(W.initializer)
sess2.run(W.initializer)
print(sess1.run(W.assign_add(10)))
print(sess2.run(W.assign_sub(2)))
print("\n")
sess1.close()
sess2.close()

100


2
4
8


20
18


20
8




### Control dependencies
tf.Graph.control_dependencies(control_inputs)
```
# defines which ops should be run first
# your graph g have 5 ops: a, b, c, d, e
g = tf.get_default_graph()
with g.control_dependencies([a, b, c]):
	# 'd' and 'e' will only run after 'a', 'b', and 'c' have executed.
	d = ...
	e = …


```

# Placeholder

**tf.placeholder(dtype, shape=None, name=None)**
* use `tf.Variable` for trainable variables such as weights and bias
* use `tf.placeholder` for actual traininig examples

In [6]:
a = tf.placeholder(tf.float32, shape=[3])
b = tf.constant([5,5,5], tf.float32)
c = a + b
with tf.Session() as sess:
    print(sess.run(c, feed_dict={a:[1,2,3]})) # tensor a is the key, not string 'a'

[6. 7. 8.]


**What if want to feed multiple data points in?**

Have to do it one at a time
```
with tf.Session as sess:
    for a_value in list_of_values_for_a:
        print(sess.run(c, {a: a_value}))
```

**You can `feed_dict` any feedable tensor**
```
tf.Graph.is_feedable(tensor)
```

In [7]:
# feeding values to TF ops
# extremely helpful for testing
a = tf.add(2,5)
b = tf.multiply(a,3)
with tf.Session() as sess:
    print(sess.run(b, feed_dict={a:15}))

45


# Lazy loading (bad idea)

Normal loading:
```
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y) 		# create the node before executing the graph

writer = tf.summary.FileWriter('./graphs/normal_loading', tf.get_default_graph())
with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())
	for _ in range(10):
		sess.run(z)
writer.close()
```

Lazy loading:
```
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

writer = tf.summary.FileWriter('./graphs/normal_loading', tf.get_default_graph())
with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())
	for _ in range(10):
		sess.run(tf.add(x, y)) # someone decides to be clever to save one line of code
writer.close()


```

With lazy loading, tf.add(x,y) added 10 times to the graph defintiion

Solution: 
* Separate definition of ops from computing/running ops 
* Use Python property to ensure function is also loaded once the first time it is called